In [1]:
import requests
import bs4
from bs4 import BeautifulSoup

import numpy  as np
import pandas as pd
import json

import os
from os import path
import re
import time

from tqdm import tqdm

DIRETORIO_PDF = 'BauruPDF/'
DIRETORIO_TXT = 'BauruTXT/'
DIRETORIO_PAR = 'BauruPAR/'
DIRETORIO_DOC = 'BauruDOC/'
DIRETORIO_JSON = 'BauruJSON/'

In [2]:
def obter_links_diarios_oficiais(ano, mes):
    assert type(ano) == str
    assert type(mes) == str
    
    lista_links = []
    
    resposta = requests.get(f'https://www2.bauru.sp.gov.br/juridico/diariooficial.aspx?a={ano}&m={mes}')
    if resposta.status_code != 200:
        print('Página retornou erro', resposta.status_code)
        return
    
    soup = BeautifulSoup(resposta.text, 'html.parser')
    
    itens = soup.find_all('li')
    for item in itens:
        ancora = item.find('a')
        if ancora != None:
            link = ancora['href']
            if path.splitext(link)[1].lower() == '.pdf':
                lista_links.append(link)
    
    return lista_links

def obter_diario_oficial(link, diretorio_destino = DIRETORIO_PDF):
    assert type(link) == str
    nome_arquivo = path.basename(link)
    assert not path.isfile(path.join(diretorio_destino, nome_arquivo))
    
    resposta = requests.get(f'https://www2.bauru.sp.gov.br{link}')
    if resposta.status_code != 200:
        print('Página retornou erro', resposta.status_code)
        return
    
    with open(path.join(diretorio_destino, nome_arquivo), 'wb') as arquivo:
        arquivo.write(resposta.content)
        
    return nome_arquivo

def obter_lista_de_dos(links, diretorio_destino = DIRETORIO_PDF, tempo_espera = 0.1):
    arquivos = []
    for link in tqdm(links):
        nome = obter_diario_oficial(link, diretorio_destino = diretorio_destino)
        arquivos.append(nome)
        time.sleep(tempo_espera)
    
    return arquivos

In [3]:
def obter_tabela_licitacoes(t = 1):
    """
    Obtém os dados resumidos da página de licitações abertas.

    :keyword t int: id interno da tabela.
        * 'Licitações Abertas' se t = 1;
        * 'Licitações Suspensas' se t = 2;
        * 'Licitações Encerradas' se t = 3.

    :returns: DataFrame com os dados da tabela
    """
    
    # Obter a página de:

    resposta = requests.get(f'https://www2.bauru.sp.gov.br/administracao/licitacoes/licitacoes.aspx?t={t}')
    if resposta.status_code != 200:
        print('Página retornou erro', resposta.status_code)
        return
    
    soup = BeautifulSoup(resposta.text, 'html.parser')
    
    # Buscar a tabela das licitações e obter o corpo
    licitacao_table = soup.find('table')
    # Não há dados relevantes no cabeçalho da tabela.
    #cabecalho_tabela = licitacao_table.thead
    conteudo_tabela = licitacao_table.tbody
    
    # Para cada linha (child em conteudo_tabela, ou <tr />) da tabela,
    #   extrair as três colunas (contents[0..2], <td />)
    licitacoes = []
    extrator_numero_ano = re.compile('(?P<numero>\d+)/(?P<ano>\d{2,4})')
    extrator_link       = re.compile('licitacoes_detalhes\.aspx\?l\=(?P<numero_link>\d+)')
    
    for child in conteudo_tabela:
        objeto       = child.contents[0].a.text
        modalidade   = child.contents[1].a.contents[0]

        numero_re = extrator_numero_ano.search(child.contents[1].a.contents[2])
        modalidade_numero = numero_re.group('numero')
        ano = numero_re.group('ano')

        interessados = child.contents[2].a.text

        link_re = extrator_link.search(child.contents[0].a['href'])
        numero_link = link_re.group('numero_link')

        licitacoes.append({'modalidade': modalidade,
                           'modalidade_numero': modalidade_numero,
                           'ano': ano,
                           'interessado': interessados,
                           'objeto': objeto,
                           'link': numero_link})

    df_licitacoes = pd.DataFrame(licitacoes)
    return df_licitacoes

In [4]:
extrator_numero_ano = re.compile('((?P<numero>\d{1,3}(\.\d{3})?)/(?P<ano>\d{2,4}))')
extrator_nome_numero_ano = re.compile('((?P<nome>\D+)\s+(?P<numero>\d{1,3}(\.\d{3})?)/(?P<ano>\d{2,4}))')
extrator_anexo = re.compile('Anexo (?P<numero>\d+) - (?P<descricao>.*)')
extrator_publicacao = re.compile('(?P<dia>\d{1,2})/(?P<mes>\d{1,2})/(?P<ano>\d{2,4})\s*:\s*(?P<titulo>(\s?\w)+)\s*:')
extrator_data = re.compile('(?P<hora>\d{1,2}):(?P<minuto>\d{1,2}) horas do dia (?P<dia>\d{1,2}) de (?P<mes>\w+) de (?P<ano>\d{2,4}) \((\w+-feira|sábado|domingo)\)')

def extrair_data(s):
    resultado = extrator_data.search(s)
    if type(resultado) != None:
        return {
            'ano': resultado.group('ano'),
            'mes': resultado.group('mes'),
            'dia': resultado.group('dia'),
            'hora': resultado.group('hora'),
            'minuto': resultado.group('minuto')
        }
    else:
        return s

def obter_detalhes_licitacao(identificador):
    """
    Obtém os dados detalhados de uma licitação a partir do código interno do site
    
    :param identificador int: código indentificador interno do site
    :returns: dict com os dados extraídos
    """
    
    # Adquirir a página de uma licitação
    response = requests.get(f'https://www2.bauru.sp.gov.br/administracao/licitacoes/licitacoes_detalhes.aspx?l={identificador}')
    if response.status_code != 200:
        print('Página retornou erro', response.status_code)
        return
    soup = BeautifulSoup(response.text, 'html.parser')
    
    conteudo = soup.find('main').find('div', class_ = 'col-10')

    detalhes_dict = {
        'identificador': identificador
    }

    for linha in conteudo.find_all('div', class_ = 'row'):
        if len(linha.contents) == 1:
            resultado = extrator_nome_numero_ano.search(linha.text)
            detalhes_dict['titulo'] = {
                'modalidade': resultado.group('nome'),
                'numero': resultado.group('numero'),
                'ano':    resultado.group('ano')
            }

        elif len(linha.contents) == 2:
            indice = linha.find('div', class_ = 'col-md-2').text
            valores = linha.find('div', class_ = 'col-md-10')

            if indice == 'Tipo:':
                indice = 'tipo'
                detalhes_dict[indice] = valores.text
                
            elif indice == 'Interessado:':
                indice = 'interessado'
                detalhes_dict[indice] = valores.text
                
            elif indice == 'Processo:':
                indice = 'processo'
                resultado = extrator_numero_ano.search(valores.text)
                detalhes_dict[indice] = {
                    'numero': resultado.group('numero'),
                    'ano': resultado.group('ano')
                }
                
            elif indice == 'Especificação:':
                indice = 'especificacao'
                detalhes_dict[indice] = valores.text
                
            elif indice == 'Prazo para Recebimento Propostas:':
                indice = 'prazo_recebimento_propostas'
                detalhes_dict[indice] = extrair_data(valores.text)
                
            elif indice == 'Prazo para Apresentação de Propostas:':
                indice = 'prazo_apresentacao_propostas'
                detalhes_dict[indice] = extrair_data(valores.text)
                
            elif indice == 'Prazo para Entrega dos Envelopes:':
                indice = 'prazo_entrega_envelopes'
                detalhes_dict[indice] = extrair_data(valores.text)

            elif indice == 'Processo Tribunal de Contas:':
                indice = 'processo_tribunal_de_contas'
                detalhes_dict[indice] = valores.text
                
            elif indice == 'Data de vencimento:':
                indice = 'data_vencimento'
                detalhes_dict[indice] = extrair_data(valores.text)
                
            elif indice == 'Data:':
                indice = 'data'
                detalhes_dict[indice] = extrair_data(valores.text)
            
            elif indice == 'Observação:':
                indice = 'observacao'
                detalhes_dict[indice] = valores.text
                
            elif indice == 'Processo Apensado' or indice == 'Processos Apensados':
                indice = 'processos_apensados'
                
                if 'processos_apensados' not in detalhes_dict:
                    detalhes_dict['processos_apensados'] = []
                
                matches = extrator_numero_ano.finditer(valores.text)
                for p in matches:
                    detalhes_dict['processos_apensados'].append({
                        'numero': p.group('numero'),
                        'ano': p.group('ano')
                    })
                
            elif indice == 'Documentos:':
                indice = 'documentos'

                if 'documentos' not in detalhes_dict:
                    detalhes_dict['documentos'] = []

                for d in valores.find_all('li'):
                    if d.b.string[0] == 'E':
                        # Assume 'Edital xxx/xxxx'
                        resultado = extrator_nome_numero_ano.search(d.b.string)
                        detalhes_dict['documentos'].append({
                            'nome': resultado.group('nome'),
                            'numero': resultado.group('numero'),
                            'ano': resultado.group('ano'),
                            'link': d.a['href']
                        })
                    elif d.b.string[0] == 'A':
                        # Assume 'Anexo x - [...]'
                        resultado = extrator_anexo.search(d.b.string)
                        detalhes_dict['documentos'].append({
                            'nome': 'Anexo',
                            'numero': resultado.group('numero'),
                            'descricao': resultado.group('descricao'),
                            'link': d.a['href']
                        })
                    else:
                        # Situação inesperada??
                        print('Situação Inesperada analisando documentos. "Anexo" ou "Edital" não encontrados.', detalhes_dict['titulo'])
                        detalhes_dict['documentos'].append((d.b.string, d.a['href']))

            elif indice == 'Publicações:':
                indice = 'publicacoes'

                if 'publicacoes' not in detalhes_dict:
                    detalhes_dict['publicacoes'] = []

                for i in range(0, len(valores.contents), 3):
                    resultado = extrator_publicacao.search(valores.contents[i].string)
                    publicacao = {
                        'titulo': resultado.group('titulo'),
                        'dia': resultado.group('dia'),
                        'mes': resultado.group('mes'),
                        'ano': resultado.group('ano'),
                        'conteudo': valores.contents[i + 1]
                    }
                    detalhes_dict['publicacoes'].append(publicacao)
                    
            else:
                if 'misc' not in detalhes_dict:
                    detalhes_dict['misc'] = []
                
                detalhes_dict['misc'].append((indice, str(valores)))
                
            
        else:
            print('quantidade de valores inesperado:')
            print(linha)
    
    return detalhes_dict

In [10]:
licitacoes_df = obter_tabela_licitacoes(t = 3)

In [5]:
def baixar_licitacoes(df, tempo_espera = 0.1, diretorio = DIRETORIO_JSON):
    """
    Baixa todos as licitações num dataframe
    
    :param df DataFrame: DataFrame com todas as licitações a baixar. Será utilizado a Series 'link'.
    :keyword tempo_espera int: Tempo de espera entre o download de uma licitação e outra. Utilizado para não enviar muitas requisições em um curto período de tempo.
    :keyword diretorio str: Diretorio de destino para os arquivos .json baixados.
    """
    
    for link_licitacao in tqdm(df['link'].values):
        #assert path.isfile(path.join(diretorio, link_licitacao + '.json')) == False,      f'Arquivo {link_licitacao}.json já existe'
        if path.isfile(path.join(diretorio, link_licitacao + '.json')):
            continue
        
        dict_licitacao = obter_detalhes_licitacao(link_licitacao)
        
        with open(path.join(diretorio, link_licitacao + '.json'), 'w') as arquivo:
            json.dump(dict_licitacao, arquivo, ensure_ascii = False)

        time.sleep(tempo_espera)

In [12]:
baixar_licitacoes(licitacoes_df[licitacoes_df['ano'] == '2022'])
baixar_licitacoes(licitacoes_df[licitacoes_df['ano'] == '2021'])
baixar_licitacoes(licitacoes_df[licitacoes_df['ano'] == '2020'])
baixar_licitacoes(licitacoes_df[licitacoes_df['ano'] == '2019'])
baixar_licitacoes(licitacoes_df[licitacoes_df['ano'] == '2018'])
baixar_licitacoes(licitacoes_df[licitacoes_df['ano'] == '2017'])
baixar_licitacoes(licitacoes_df[licitacoes_df['ano'] == '2016'])

100%|██████████| 311/311 [02:32<00:00,  2.04it/s]


In [45]:
links = []
for m in ['12', '11', '10', '09', '08', '07', '06', '05', '04', '03', '02', '01']:
    links_ = obter_links_diarios_oficiais('2016', m)
    links.extend(links_)

arquivos = obter_lista_de_dos(links)
print(arquivos)

100%|██████████| 147/147 [05:45<00:00,  2.35s/it]

['do_20161231_2773.pdf', 'do_20161229_2772.pdf', 'do_20161227_2771.pdf', 'do_20161224_2770.pdf', 'do_20161222_2769.pdf', 'do_20161220_2768.pdf', 'do_20161217_2767.pdf', 'do_20161215_2766.pdf', 'do_20161213_2765.pdf', 'do_20161210_2764.pdf', 'do_20161208_2763.pdf', 'do_20161206_2762.pdf', 'do_20161203_2761.pdf', 'do_20161201_2760.pdf', 'do_20161129_2759.pdf', 'do_20161126_2758.pdf', 'do_20161124_2757.pdf', 'do_20161122_2756.pdf', 'do_20161119_2755.pdf', 'do_20161117_2754.pdf', 'do_20161112_2753.pdf', 'do_20161110_2752.pdf', 'do_20161108_2751.pdf', 'do_20161105_2750.pdf', 'do_20161101_2749.pdf', 'do_20161027_2748.pdf', 'do_20161025_2747.pdf', 'do_20161022_2746.pdf', 'do_20161020_2745.pdf', 'do_20161018_2744.pdf', 'do_20161015_2743.pdf', 'do_20161013_2742.pdf', 'do_20161011_2741.pdf', 'do_20161008_2740.pdf', 'do_20161006_2739.pdf', 'do_20161004_2738.pdf', 'do_20161001_2737.pdf', 'do_20160929_2736.pdf', 'do_20160927_2735.pdf', 'do_20160924_2734.pdf', 'do_20160922_2733.pdf', 'do_20160920_27

In [25]:
obter_diario_oficial('/arquivos/sist_diariooficial/2022/06/do_20220614_3563.pdf')

AssertionError: 